# Best Routes Extractor (Top Profit per Day)

This notebook scans a folder of CSV files named like:

- `routes_<HUB>_<AIRCRAFT>_<TRIPS>.csv`  
  Example: `routes_LIM_a388_2.csv`

It reads **only the first N lines per file** (default 50), computes:

- **profit_per_day** = `profit_pt * trips_pd_pa`
- **hours_per_trip** = `24 / trips_pd_pa`

Then exports the **Top K routes** (default 500) to a single CSV.

---


In [ ]:
# Install / import dependencies
# If needed: !pip install pandas

from __future__ import annotations

import re
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Iterable, Tuple

import pandas as pd

In [ ]:
!pip install numpy pandas

In [ ]:
# Configuration (edit these values, or call functions directly)

BASE_FOLDER = Path.cwd()
INPUT_FOLDER = BASE_FOLDER  # change to your folder
AIRCRAFT_FILTER = "a388"            # e.g. "a388" or None
TRIPS_FILTER = None                 # e.g. 3 or None
LINES_PER_FILE = 50                 # first N lines only
TOP_K = 500                         # best routes to keep
OUTPUT_CSV = BASE_FOLDER / f"best_routes_{AIRCRAFT_FILTER}.csv"


In [ ]:
# Core functions

FILENAME_RE = re.compile(
    r"^routes_(?P<hub>[^_]+)_(?P<aircraft>[^_]+)_(?P<trips>\d+)\.csv$",
    re.IGNORECASE,
)

REQUIRED_COLUMNS = [
    "dest.id",
    "dest.name",
    "dest.country",
    "stop.name",
    "stop.country",
    "cfg.y",
    "cfg.j",
    "cfg.f",
    "tkt.y",
    "tkt.j",
    "tkt.f",
    "trips_pd_pa",
    "profit_pt",
]

OUTPUT_COLUMNS = [
    "HUB",
    "dest.id",
    "dest.name",
    "dest.country",
    "stop.name",
    "stop.country",
    "cfg.y",
    "cfg.j",
    "cfg.f",
    "tkt.y",
    "tkt.j",
    "tkt.f",
    "hours_per_trip",   # 24 / trips_pd_pa
    "profit_per_day",   # profit_pt * trips_pd_pa
]


@dataclass(frozen=True)
class RouteFileMeta:
    path: Path
    hub: str
    aircraft: str
    trips_in_name: int


def iter_route_files(
    folder: Path,
    aircraft_filter: Optional[str] = None,
    trips_filter: Optional[int] = None,
) -> Iterable[RouteFileMeta]:
    """Yield metadata for all matching routes_*.csv files in folder."""
    folder = Path(folder)
    for p in folder.iterdir():
        if not p.is_file():
            continue
        m = FILENAME_RE.match(p.name)
        if not m:
            continue

        hub = m.group("hub")
        aircraft = m.group("aircraft")
        trips_in_name = int(m.group("trips"))

        if aircraft_filter and aircraft_filter.lower() not in p.name.lower():
            continue
        if trips_filter is not None and trips_in_name != trips_filter:
            continue

        yield RouteFileMeta(path=p, hub=hub, aircraft=aircraft, trips_in_name=trips_in_name)


def read_top_lines_csv(path: Path, nrows: int = 50) -> pd.DataFrame:
    """Read only the first `nrows` rows from a CSV."""
    return pd.read_csv(path, nrows=nrows)


def compute_profit_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """Compute hours_per_trip and profit_per_day (returns a copy)."""
    out = df.copy()

    # numeric coercion
    out["trips_pd_pa"] = pd.to_numeric(out["trips_pd_pa"], errors="coerce")
    out["profit_pt"] = pd.to_numeric(out["profit_pt"], errors="coerce")

    # drop invalid rows
    out = out.dropna(subset=["trips_pd_pa", "profit_pt"])
    out = out[out["trips_pd_pa"] != 0]

    out["hours_per_trip"] = 24.0 / out["trips_pd_pa"]
    out["profit_per_day"] = out["profit_pt"] * out["trips_pd_pa"]

    return out


def extract_best_routes(
    folder: Path,
    aircraft_filter: Optional[str] = None,
    trips_filter: Optional[int] = None,
    lines_per_file: int = 50,
    top_k: int = 500,
) -> pd.DataFrame:
    """Return a DataFrame with the top K routes by profit_per_day."""
    folder = Path(folder)
    all_rows = []
    matched_files = 0

    for meta in iter_route_files(folder, aircraft_filter=aircraft_filter, trips_filter=trips_filter):
        matched_files += 1
        try:
            df = read_top_lines_csv(meta.path, nrows=lines_per_file)
        except Exception as e:
            print(f"[WARN] Could not read {meta.path.name}: {e}")
            continue

        missing = [c for c in REQUIRED_COLUMNS if c not in df.columns]
        if missing:
            print(f"[WARN] Skipping {meta.path.name} (missing columns: {missing})")
            continue

        df = df[REQUIRED_COLUMNS].copy()
        df.insert(0, "HUB", meta.hub)

        df = compute_profit_metrics(df)
        all_rows.append(df)

    if matched_files == 0:
        raise ValueError(
            "No files matched the pattern routes_<HUB>_<AIRCRAFT>_<TRIPS>.csv "
            "with your current filters."
        )

    if not all_rows:
        raise ValueError("Files matched, but no usable rows were found (read/column issues).")

    big = pd.concat(all_rows, ignore_index=True)
    big = big.sort_values("profit_per_day", ascending=False).head(top_k)

    return big[OUTPUT_COLUMNS].copy()


def export_best_routes_to_csv(
    folder: Path,
    output_csv: Path,
    aircraft_filter: Optional[str] = None,
    trips_filter: Optional[int] = None,
    lines_per_file: int = 50,
    top_k: int = 500,
) -> Path:
    """Extract best routes and export them to a CSV. Returns output path."""
    df = extract_best_routes(
        folder=folder,
        aircraft_filter=aircraft_filter,
        trips_filter=trips_filter,
        lines_per_file=lines_per_file,
        top_k=top_k,
    )

    output_csv = Path(output_csv)
    output_csv.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(output_csv, index=False)
    return output_csv


## Example usage (run these cells)

You can call the functions from other code cells, for example:
- `extract_best_routes(...)` returns a DataFrame
- `export_best_routes_to_csv(...)` writes the CSV


In [ ]:
# Example 1: Get a DataFrame (top routes) and preview it

df_best = extract_best_routes(
    folder=INPUT_FOLDER,
    aircraft_filter=AIRCRAFT_FILTER,
    trips_filter=TRIPS_FILTER,
    lines_per_file=LINES_PER_FILE,
    top_k=TOP_K,
)

df_best.head(10)


In [ ]:
# Example 2: Export to CSV

out_path = export_best_routes_to_csv(
    folder=INPUT_FOLDER,
    output_csv=OUTPUT_CSV,
    aircraft_filter=AIRCRAFT_FILTER,
    trips_filter=TRIPS_FILTER,
    lines_per_file=LINES_PER_FILE,
    top_k=TOP_K,
)

out_path


## Notes / customization

- If you set `TRIPS_FILTER = 3`, it selects files ending with `_3.csv`.
- If you leave `TRIPS_FILTER = None`, it includes *all trips* for the selected aircraft.
- If you set `AIRCRAFT_FILTER = None`, it includes all aircraft types (based on filename).
